In [213]:
import pandas as pd
import numpy as np
import random
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import date

In [214]:
data = pd.read_csv("data/df_new.csv")

### 1. Split data for optim part

In [215]:
df_opti = data[data.ScheduledTime >= "2016-05-30"]
df_opti = df_opti[df_opti.ScheduledTime <= "2016-06-05"]

df_opti.to_csv("data/data_opti.csv", index = False)

In [216]:
df_pred = data[data.ScheduledTime < "2016-05-30"]

df_pred.to_csv("data/data_pred.csv", index = False)

### 2. Select one clinic and nb doctors

Select clinic with enough appointments and high no show rate : 5800 appointments and 21% no show for Maria Ortiz

In [217]:
neigh = "MARIA ORTIZ"

n_doc = np.floor(len(df_opti[df_opti.Neighbourhood == neigh])/(5*10))

In [218]:
n_doc = 22

In [219]:
df_pred_c = df_pred[df_pred.Neighbourhood == neigh]

In [220]:
df_opti_c = df_opti[df_opti.Neighbourhood == neigh]

### 3. Get prebooked slots

In [221]:
df_pred_c = df_pred_c[df_pred_c.AppointmentTime >= "2016-05-30"]
df_pred_c = df_pred_c[df_pred_c.AppointmentTime <= "2016-06-05"]
df_pred_c['AppointmentTime']= pd.to_datetime(df_pred_c['AppointmentTime'])
df_pred_c.AppointmentTime = df_pred_c.AppointmentTime.dt.tz_localize(None)
df_pred_c["slot"] = df_pred_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)
df_pred_c.to_csv("data/pred_Maria.csv")

In [222]:
df_opti_c = df_opti_c[df_opti_c.AppointmentTime >= "2016-05-30"]
df_opti_c = df_opti_c[df_opti_c.AppointmentTime <= "2016-06-05"]
df_opti_c.to_csv("data/opti_Maria.csv")

In [223]:
df_opti_c

,PatientId,AppointmentID,Gender,ScheduledTime,AppointmentTime,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,ScheduledDay,ScheduledMonth,ScheduledYear,ScheduledWeekDay,AppointmentHour,AppointmentDay,AppointmentMonth,AppointmentYear,AppointmentWeekDay,AppointmentHourRandom
3250,23306,95781,F,2016-05-31 11:47:10+00:00,2016-05-31 23:59:59+00:00,49,MARIA ORTIZ,0,0,0,...,31,5,2016,1,15,31,5,2016,1,16
3266,38700,93903,F,2016-05-30 19:25:44+00:00,2016-05-31 23:59:59+00:00,24,MARIA ORTIZ,0,0,0,...,30,5,2016,0,9,31,5,2016,1,11
3294,41570,94241,F,2016-05-31 07:18:33+00:00,2016-05-31 23:59:59+00:00,42,MARIA ORTIZ,0,0,0,...,31,5,2016,1,13,31,5,2016,1,13
3296,9842,91782,F,2016-05-30 09:23:34+00:00,2016-05-30 23:59:59+00:00,52,MARIA ORTIZ,0,0,0,...,30,5,2016,0,12,30,5,2016,0,15
3297,40527,91640,F,2016-05-30 09:01:04+00:00,2016-05-30 23:59:59+00:00,2,MARIA ORTIZ,0,0,0,...,30,5,2016,0,17,30,5,2016,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110499,59334,98085,F,2016-06-01 09:42:56+00:00,2016-06-01 23:59:59+00:00,66,MARIA ORTIZ,0,1,1,...,1,6,2016,2,15,1,6,2016,2,18
110501,5697,98260,M,2016-06-01 10:19:12+00:00,2016-06-01 23:59:59+00:00,44,MARIA ORTIZ,0,0,0,...,1,6,2016,2,17,1,6,2016,2,12
110505,37540,98410,F,2016-06-01 10:45:50+00:00,2016-06-01 23:59:59+00:00,55,MARIA ORTIZ,0,0,0,...,1,6,2016,2,18,1,6,2016,2,11
110506,9273,98531,M,2016-06-01 11:09:20+00:00,2016-06-01 23:59:59+00:00,5,MARIA ORTIZ,0,0,0,...,1,6,2016,2,12,1,6,2016,2,16


### 4. Generate random availability patient

In [224]:
for i in range(10):
    df_opti_c.loc[:,"Availability_"+str(i)] = [random.randint(0,51) for n in range(len(df_opti_c))]

In [225]:
df_opti_c.to_csv("data/opti_Maria.csv")

### 5. Generate day availability to compare models

In [227]:
df_opti_c['AppointmentTime']= pd.to_datetime(df_opti_c['AppointmentTime'])
df_opti_c.AppointmentTime = df_opti_c.AppointmentTime.dt.tz_localize(None)
for i in range(5):
    df_opti_c.loc[:,"Availability_day_"+str(i)] = df_opti_c.apply(
    lambda row: (row.AppointmentTime - pd.to_datetime("2016-05-30")).days*10 + random.randint(0,9), axis=1)

In [229]:
df_opti_c.to_csv("data/opti_Maria.csv")